# Reports Module

## Initialisation

Basic approach to determine the project directory

In [1]:
import os
import glob
import csv

import json
import unittest

from common import Printable, testExit, projdir

from entrant import Entrant

from constants import *

## Report Class

Class to manage a single report

In [2]:
class Report(Printable):
    def __init__(self, config, verbosity=1):
        '''Initialise report object'''    

        super().__init__(verbosity=verbosity)
        
        self.config = config.copy()
        

    def patchConfig(self, config):
        '''Patch config with overrides - e.g. period specific or global / event'''
        
        for key in config:
            # Do not copy Periods or Reports from the global config
            if key not in ['Periods', 'Reports'] and key not in self.config:
                self.config[key] = config[key]


    def parseText(self):
        '''Process the titles'''

        if 'Title' in self.config:
            self.title = self.config['Title']
        else:
            raise ValueError('Missing title for report - {}'.format(self.config))

        if 'Period' in self.config:
            self.period = self.config['Period']
        else:
            self.period = 'Session'
            
        if self.period not in ['Course', 'Session', 'Event', 'Round']:
            raise ValueError('Invalid period for report "{}"- {}'.format(self.title, self.period))
        
        if 'Subtitle' in self.config:
            self.subtitle = self.config['Subtitle']
        else:
            self.subtitle = None

        if 'Suptitle' in self.config:
            self.suptitle = self.config['Suptitle']
        else:
            self.suptitle = None

        if 'Series' in self.config:
            self.series = self.config['Series']
        else:
            self.series = None
            
        # TODO - validate series


    def parseLists(self):
        '''Process the lists'''

        if 'Fields' in self.config:
            self.fields = self.config['Fields'].split(',')
        else:
            raise ValueError('Missing field list for report "{}"'.format(self.title))

        if 'HideFields' in self.config:
            for hideField in self.config['HideFields'].split(','):
                if hideField in self.fields:
                    self.fields.remove(hideField)
                else:
                    raise ValueError('Invalid field to be hidden for report "{}" - "{}"'.format(self.title, hideField))

        if 'Filter' in self.config:
            self.filter = self.config['Filter'].split(',')
        else:
            self.filter = []

        # TODO - validate filter?

        if 'Sort' in self.config:
            self.sort = self.config['Sort'].split(',')
        else:
            self.sort = ['Speed (kts)=DSC', 'Time=ASC']

        # TODO - validate sort?


    def parseInts(self):
        '''Process the ints'''

        if 'Limit' in self.config:
            self.limit = int(self.config['Limit'])
        else:
            self.limit = 99999

        if 'Prizes' in self.config:
            self.prizes = int(self.config['Prizes'])
        else:
            self.prizes = 0

        if 'ShowSpeeds' in self.config:
            self.showSpeeds = int(self.config['ShowSpeeds'])
        else:
            self.showSpeeds = 1


    def parseBools(self):
        '''Process the booleans'''

        def getBool(key):
            '''Convert Y / N to true / false'''

            value = self.config[key]

            if value not in [True, False]:
                if value[:1].lower() == 'n':
                    result = False
                elif value[:1].lower() == 'y':
                    result = True
            else:
                result = value

            if value not in [True, False]:
                raise ValueError('Bad value for "{}"'.format(entrant.getValue('ID')))

            return result

        if 'Profile' in self.config:
            self.profile = getBool('Profile')
        else:
            self.profile = False

        if 'Rank' in self.config:
            self.rank = getBool('Rank')
        else:
            self.rank = True

        if 'Trophy' in self.config:
            self.trophy = getBool('Trophy')
        else:
            self.trophy = False

        if 'Warning' in self.config:
            self.warning = getBool('Warning')
        else:
            self.warning = False


    def parseConfig(self):
        '''Process the configuration'''

        self.parseText()
        self.parseLists()
        self.parseInts()
        self.parseBools()
        
        del(self.config)

## Reports Class

Class to manage a collection of reports

In [3]:
class Reports(Printable):
    def __init__(self, eventPath, verbosity=1):
        '''Initialise reports object'''    

        super().__init__(verbosity=verbosity)
        
        self.eventPath = eventPath
        
        self.periods = {}


    def indexReports(self, filename, reports):
        '''Index the reports by period'''
        
        # Start by creating a single list for each period
        periods = {}
        for report in reports:
            period = report.period
            if period not in periods:
                periods[period] = []
            periods[period].append(report)

        # Finish off by adding the simple lists to the main collections
        basename = os.path.splitext(os.path.basename(filename))[0]
        for period in periods:
            if period not in self.periods:
                self.periods[period] = {}
            self.periods[period][basename] = periods[period]

        
    def loadReports(self):
        '''Load report configurations from JSON files'''
        
        filenames = glob.glob(os.path.join(os.path.join(self.eventPath, REPORTS_DIR), '*.json'))
        
        for filename in filenames:
            with open(filename, 'r', encoding='utf-8') as f:
                jsonTxt = f.read()
                try:
                    reportsConfig = json.loads(jsonTxt)
                except:
                    self.logError('Could not parse {}'.format(filename))
                    raise

                reports = []

                for reportConfig in reportsConfig['Reports']:
                    if 'Periods' in reportsConfig:
                        for periodConfig in reportsConfig['Periods']:
                            if 'Period' not in reportConfig or reportConfig['Period'] == periodConfig['Period']:
                                report = Report(reportConfig)
                                report.patchConfig(periodConfig)
                                report.patchConfig(reportsConfig)
                                report.parseConfig()                           
                                reports.append(report)
                    else:
                        report = Report(reportConfig)
                        report.patchConfig(reportsConfig)
                        report.parseConfig()
                        reports.append(report)

                self.indexReports(filename, reports)

## Unit Tests

A handful of basic filter tests, utilising a dummy "event" class

In [4]:
class DummyEvent(Printable):
    def __init__(self, path, verbosity=1):
        
        super().__init__(verbosity=verbosity)

        self.path = path

        self.entrants = {}
        
        self.reports = Reports(self.path)


    def loadReports(self):
        '''Read reports from JSON files'''

        self.reports.loadReports()


    def loadEntrants(self):
        '''Read entrants from JSON'''

        csvPath = os.path.join(self.path, CONFIG_DIR, ENTRANTS_CSV)
        
        with open(csvPath, 'r', encoding='utf-8') as f:
            csvReader = csv.reader(f)
            headers = next(csvReader)

            for values in csvReader:
                entrant = Entrant(headers, values, verbosity=self.verbosity)
                if entrant.getValue('ID') not in self.entrants:
                    self.entrants[entrant.getValue('ID')] = entrant
                else:
                    raise ValueError('Duplicate entrant ID "{}"'.format(entrant.getValue('ID')))

        self.logInfo('{} entrants loaded'.format(len(self.entrants)))

In [5]:
class TestReports1998(unittest.TestCase):
    '''Class to test reports using 2019 data'''

    def testPeriods(self):
        '''Test the periods have been indexed'''

        periods = event.reports.periods.keys()
        self.assertEqual('Session' in periods, True)
        self.assertEqual('Event' in periods, True)


    def testSessionReports(self):
        '''Test the session reports have been indexed'''

        reports = event.reports.periods['Session'].keys()
        self.assertEqual('fastest' in reports, True)
        self.assertEqual('runs' in reports, True)
        self.assertEqual('verification' in reports, True)
        self.assertEqual('craft' in reports, True)


    def testEventReports(self):
        '''Test the event reports have been indexed'''

        reports = event.reports.periods['Event'].keys()
        self.assertEqual('fastest' in reports, True)
        self.assertEqual('entrants' in reports, True)
        self.assertEqual('craft' in reports, True)


    def testSessionFastest(self):
        '''Test the session fastest'''

        fastest = event.reports.periods['Session']['fastest']
        self.assertEqual(len(fastest), 4)

        for report in fastest:
            self.assertEqual(report.period, 'Session')
            self.assertEqual('Name' in report.fields, True)
            self.assertEqual('Course' in report.fields, True)


    def testEventFastest(self):
        '''Test the event fastest'''

        fastest = event.reports.periods['Event']['fastest']
        self.assertEqual(len(fastest), 4)

        for report in fastest:
            self.assertEqual(report.period, 'Event')
            self.assertEqual('Name' in report.fields, True)
            self.assertEqual('Course' in report.fields, False)


    def testSuptitles(self):
        '''Test the suptitles'''

        fastest = event.reports.periods['Session']['fastest'] + event.reports.periods['Event']['fastest']
        for report in fastest:
            self.assertNotEqual(report.suptitle, None)


    def testSubtitles(self):
        '''Test the subtitles'''

        fastest = event.reports.periods['Session']['fastest'] + event.reports.periods['Event']['fastest']
        for report in fastest:
            if report.title.endswith('Overall'):
                self.assertEqual(report.subtitle, None)
            else:
                self.assertNotEqual(report.subtitle, None)


    def testFilters(self):
        '''Test the filters'''

        fastest = event.reports.periods['Session']['fastest'] + event.reports.periods['Event']['fastest']
        for report in fastest:
            if report.title.endswith('Overall'):
                self.assertEqual(report.filter, [])
            else:
                self.assertNotEqual(report.filter, [])


    def testLimits(self):
        '''Test the limits'''

        fastest = event.reports.periods['Session']['fastest'] + event.reports.periods['Event']['fastest']
        for report in fastest:
            if report.title.endswith('Overall'):
                self.assertEqual(report.limit, 10)
            else:
                self.assertEqual(report.limit, 3)


    def testWarnings(self):
        '''Test the warnings'''

        fastest = event.reports.periods['Session']['fastest'] + event.reports.periods['Event']['fastest']
        for report in fastest:
            if 'Unclassified' in report.title:
                self.assertEqual(report.warning, True)
            else:
                self.assertEqual(report.warning, False)

## Run Unit Tests

Note: Only run unit tests when running this script directly, not during an import

In [6]:
if __name__ == '__main__':
    # Read main config into global variable
    filename = os.path.join(projdir, CONFIG_DIR, APP_CONFIG)
    with open(filename, 'r', encoding='utf-8') as f:
        jsonTxt = f.read()
        appConfig = json.loads(jsonTxt)

    # Only load the 1998 event once for the unit tests
    eventPath = os.path.join(projdir, EVENTS_DIR, '1998')
    event = DummyEvent(eventPath)
    event.loadEntrants()
    event.loadReports()

    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

..........
----------------------------------------------------------------------
Ran 10 tests in 0.020s

OK


## All Done!